In [1]:
import csv
import numpy as np
import time

import torch
from torch import nn
from torch import optim
import torchvision
from PIL import Image

In [2]:
def read_csv(train_test):
  lines = []
  #with open(f"gdrive/Shareddrives/IC - Datasets/dataset/lens/CogentAnnotation{train_test}.csv", "r") as file:
  with open(f"lens/CogentAnnotation{train_test}.csv", "r") as file:
      
    csvreader = csv.reader(file)

    for row in csvreader:
        lines.append(np.asarray(row))

  lines.pop(0) #Removing the header of the file
  lines = np.array(lines).T #Transposing to recover the data easier

  return lines

def label_classifier(labels):
    #Changing the string values to numerical using the item value to identify in the dictionary the corresponding item
    type_labels = { "Colored": 0, "Normal": 1, "Transparent": 2 }

    num_labels = len(labels)
    for i in range (0, num_labels):
        labels[i] = type_labels[labels[i]]

    return labels.astype(np.int32)

def read_images(train_test):

    #Reading csv to get image path transposed to recover easier the data
    csv_info = read_csv(train_test)

    images_path = csv_info[0]
    labels = label_classifier(csv_info[1])
    
    images = []

    for image in images_path:
        img = Image.open("lens/" + image + ".bmp")
        img = img.resize((300, 300))
        images.append(torchvision.transforms.functional.to_tensor(img))

    labels = np.array(labels)

    #Shuffle all the data
    ts = int(time.time())
    np.random.seed(ts)
    np.random.shuffle(images)
    np.random.seed(ts)
    np.random.shuffle(labels)

    return images, labels
    

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

torch.cuda.empty_cache()

cuda


In [4]:
x_train_read, y_train_read = read_images("Train")
x_test_read, y_test_read = read_images("Test")

print(x_train_read[0].size())

torch.Size([1, 300, 300])


In [5]:
x_train = (torch.stack(x_train_read)).type(torch.FloatTensor)
y_train = torch.from_numpy(y_train_read).type(torch.LongTensor)

x_test = torch.stack(x_test_read).type(torch.FloatTensor)
y_test = torch.from_numpy(y_test_read).type(torch.LongTensor)

x_train = x_train.repeat(1, 3, 1, 1)
x_test = x_test.repeat(1, 3, 1, 1)


In [6]:
print(x_test.shape)
print(x_train.shape)

torch.Size([1755, 3, 300, 300])
torch.Size([1753, 3, 300, 300])


In [7]:
from torch.utils.data import DataLoader, TensorDataset

train = TensorDataset(x_train, y_train)
test = TensorDataset(x_test, y_test)

train_loaded = DataLoader(train, batch_size=5, shuffle=True)
test_loaded = DataLoader(test, batch_size=5, shuffle=True)


In [8]:
torch.cuda.empty_cache()
class lensModel(nn.Module):
    def __init__(self):
        super(lensModel, self).__init__()

        # Stage 1-9: efficientNetB3 with resolution 300x300, 3 channels, and 1 layer
        self.weights = torchvision.models.EfficientNet_B3_Weights.DEFAULT
        self.efficientNetB3 = torchvision.models.efficientnet_b3(weights= self.weights)
        self.efficientNetB3._avg_pooling = nn.Identity()
        
        # Stage 10: BatchNormalization and Dropout with 10x10 resolution, 1536 channels, and 1 layer
        self.stage10_layers = nn.Sequential(
            nn.BatchNorm1d(1000),
            nn.Dropout(p=0.1)
        )

        # Stage 11: Fully connected layer, BatchNormalization, activation, Dropout, resolution 1, 512 channels, and 1 layer
        self.stage11_layers = nn.Sequential(
            nn.Linear(1000, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(p=0.1)
        )

        # Stage 12: Fully connected layer, BatchNormalization, activation, resolution 1, 512 channels, and 1 layer
        self.stage12_layers = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.1)
        )

        # Stage 13: Fully connected layer connected to Softmax, resolution 1, no channels, and 1 layer
        self.stage13_layers = nn.Sequential(
            nn.Linear(256, 3),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.efficientNetB3(x)
        x = self.efficientNetB3._avg_pooling(x)
        x = x.view(x.size(0), -1)

        x = self.stage10_layers(x)
        x = self.stage11_layers(x)
        x = self.stage12_layers(x)
        x = self.stage13_layers(x)

        return x
    
#Putting the MnistModel into a variable
model = lensModel()
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.91)
criterion = nn.CrossEntropyLoss()

In [9]:
from tqdm import tqdm


num_epochs = 5

train_loss_history = []
test_loss_history = []
train_acc_history = []
test_acc_history = []

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0
    train_correct = 0
    
    progress_bar = tqdm(enumerate(train_loaded), total=len(train_loaded))
    
    for batch_idx, (inputs, labels) in progress_bar:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
        train_correct += (predicted == labels).sum().item()
        
        # Update progress bar description
        progress_bar.set_description(f'Epoch [{epoch+1}/{num_epochs}] '
                                        f'Loss: {loss.item():.4f} '
                                        f'Acc: {(predicted == labels).sum().item() / len(inputs):.4f}')
    
    train_loss = train_loss / len(train_loaded.dataset)
    train_acc = train_correct / len(train_loaded.dataset)
    train_loss_history.append(train_loss)
    train_acc_history.append(train_acc)
    
    # Validation phase
    model.eval()
    test_loss = 0.0
    test_correct = 0
    
    with torch.no_grad():
        for inputs, labels in test_loaded:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            loss = criterion(outputs, labels)
            
            test_loss += loss.item() * inputs.size(0)
            test_correct += (predicted == labels).sum().item()
    
    test_loss = test_loss / len(test_loaded.dataset)
    test_acc = test_correct / len(test_loaded.dataset)
    test_loss_history.append(test_loss)
    test_acc_history.append(test_acc)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], '
            f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
            f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}\n')

Epoch [1/5] Loss: 0.9618 Acc: 0.3333: 100%|██████████| 351/351 [01:27<00:00,  4.01it/s]


Epoch [1/5], Train Loss: 1.0447, Train Acc: 0.4837, Test Loss: 0.9326, Test Acc: 0.6262



Epoch [2/5] Loss: 1.0920 Acc: 0.3333: 100%|██████████| 351/351 [01:20<00:00,  4.37it/s]


Epoch [2/5], Train Loss: 0.9048, Train Acc: 0.6361, Test Loss: 0.8469, Test Acc: 0.7254



Epoch [3/5] Loss: 0.9448 Acc: 0.3333: 100%|██████████| 351/351 [01:18<00:00,  4.48it/s]


Epoch [3/5], Train Loss: 0.8385, Train Acc: 0.7136, Test Loss: 0.7916, Test Acc: 0.7687



Epoch [4/5] Loss: 1.3005 Acc: 0.3333: 100%|██████████| 351/351 [01:16<00:00,  4.58it/s]


Epoch [4/5], Train Loss: 0.7984, Train Acc: 0.7496, Test Loss: 0.7769, Test Acc: 0.7761



Epoch [5/5] Loss: 0.6169 Acc: 1.0000: 100%|██████████| 351/351 [01:15<00:00,  4.63it/s]


Epoch [5/5], Train Loss: 0.7440, Train Acc: 0.8038, Test Loss: 0.7110, Test Acc: 0.8473



In [10]:
#Getting the Final Accuracy of the model
import torch.nn.functional as F

model.eval()
t_loss = 0
correct = 0
with torch.no_grad():
    for imgs, labels in tqdm(test_loaded, desc="Getting accuracy"):
        labels = labels.type(torch.LongTensor)
        imgs, labels = imgs.to(device), labels.to(device)
        output = model(imgs)
        t_loss += F.cross_entropy(output, labels, reduction="sum").item()

        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()

t_loss /= len(test_loaded.dataset)

print( f"Test set: Average loss: {t_loss}, Accuracy: {correct}/{len(test_loaded.dataset)}, {100.0 * correct / len(test_loaded.dataset)})\n")

Getting accuracy: 100%|██████████| 351/351 [00:19<00:00, 17.96it/s]

Test set: Average loss: 0.710961740580719, Accuracy: 1487/1755, 84.72934472934473)



In [11]:
#Saving the model and the model parameters
file_name = input("Give a name to salve the model and the optimizer: ")
torch.save(model.state_dict(),f"Modelos/{file_name}-.pth")
torch.save(optimizer.state_dict(), f"Modelos/{file_name}-optimizer.pth")
print("Model Saved")

Model Saved
